# Description of the idea 

   In this project we will be doing an exploratory analysis to find the best districts in Costa Rica to place 10 locals of a new chain restaurant in the country to do this we are basing our criteria on a market principle of placing similar business together, this is really important since our chain will be new in our country so we’ll like to be near where people are going to eat so we can pop up as a new alternative for them. 
   
   We already know food courts are a good option, but recently the looking for a culinary experience, beyond of what you find in a food court, has started to grow and with it some kind of food markets with different restaurant options, this trend has been developed really organically which means the communities don't plan to have this kind of markets near them but, some restaurants and coffees started migrating to those places to offer a more unique experience to their customers. 
    
   Since these markets are not like projects scheduled to develop per se we cannot find them strictly on the internet like food markets so we will leverage on data to find those places where these markets have started to grow and develop.

# Methodology 

To accomplish these there are some pieces of information well need to find, so the methodology and approach for the study is this:
 
   Since Costa Rica is a really small country, and our beaches are really popular around the world we will not reduce our scope only to the GAM (Big Metropolitan Area), we will study all urban and semi-urban cities as a possibility so this will let plan ahead our expansion. So, we’ll start by creating a table with all the geographical information of our cities. One thing to take in consideration is that for our Country this information is not as easy to find compared to cities in the USA, so we'll need to do some research.

   Once we have the location data well do exploratory research for venues near each city with the Four Square API. We organize this data and preprocess it to use a clustering method and find those districts which have a similitude based on food and recreation places. 
